## Done using Parameters tuning For AWR  since in the inital stage got contarcting reulst fro both geraient boosting and GBC:
## Hence to evluate results more precisely doign parameter tuning.

In [1]:
from sktime.datasets import load_UCR_UEA_dataset
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sktime.datatypes._panel._convert import from_nested_to_3d_numpy
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from pyts.datasets import load_basic_motions
from sklearn.model_selection import GridSearchCV

/Users/dinsind/.local/lib/python3.8/site-packages/sktime/datatypes/_series/_check.py:43: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  VALID_INDEX_TYPES = (pd.Int64Index, pd.RangeIndex, pd.PeriodIndex, pd.DatetimeIndex)
/Users/dinsind/.local/lib/python3.8/site-packages/sktime/datatypes/_hierarchical/_check.py:50: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  VALID_INDEX_TYPES = (pd.Int64Index, pd.RangeIndex, pd.PeriodIndex, pd.DatetimeIndex)
/Users/dinsind/.local/lib/python3.8/site-packages/sktime/datatypes/_hierarchical/_check.py:51: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  VALID_MULTIINDEX_TYPES = (pd.Int64Index, pd.RangeIndex)
/Users/dinsind/.local/lib/pyth

In [2]:
X,y= load_UCR_UEA_dataset('ArticularyWordRecognition', split="train", return_X_y=True)

In [3]:
X_test,y_test= load_UCR_UEA_dataset('ArticularyWordRecognition', split="test", return_X_y=True)

In [4]:
X_3d = from_nested_to_3d_numpy(X)
X_3d.shape

(275, 9, 144)

In [5]:
X_test_3d= from_nested_to_3d_numpy(X_test)
X_test_3d.shape

(300, 9, 144)

In [6]:
class reservoir_features:
    '''
    creates an object associated with a multivariate dataset
    
    
    '''
    def __init__(self,data,num_features):
        '''
        data: Takes in a multidimensional array (x * y * z) - z>y
        Initializes it
        x: Timeseries
        y: Attributes for a given timeseries observation
        z: timestamped observations (features)
        
        num_features: you must specify the dimension you want to reduce it to
        
        '''
        self.features = []
        self.filters_used = []
        self.original_data = data.copy()
        self.data = data.copy()
        self.num_features = num_features
        self.x = data.shape[0]
        self.y = data.shape[1]
        self.z = data.shape[2]
        # perform checks 
        #1. 3d numpy array
        #2. Each time series should have same number of observations
        #3. num_features should be less than timestamped observations
    
    def normalize(self):
        '''
        Each attribute could potentially be on a different scale
        modifies the original data and performs a min max normalization
        '''
        for i in range(self.original_data.shape[0]):
            for j in range(self.original_data.shape[1]):
                self.data[i][j] = (self.original_data[i][j] - self.original_data[i][j].min())/(self.original_data[i][j].max()-self.original_data[i][j].min())
    
    
    def filters(self,stride_len = [1], num_filters = 1):
        '''
        stride_len: num of columns to skip after each filter multiplication
        num_filters: you can specify the number of filters you need; each filter will be of a differnt size
        size of filter = n*m 
        (n = # of rows = attribute size, 
        m = # of columns)
        '''
        #Have error check to make sure stride len is a list and value is <length of attributes
        n = self.y
        
        
        #Edge case vals is empty/smaller than num_filters
        
        for iteration in range(num_filters):
            m = self._get_m(stride_len[iteration])
            filter_a = np.random.random((n,m))
            print("filter of size ", str(n), "*", str(m), "was created\n")
            self.filters_used.append(filter_a)
            
            temp_features =[]
            for i in range(self.x):
                temp = []
                j = 0
                while j + m < self.data.shape[2]:
                    temp.append((filter_a*self.data[i,:,j:j+m]).mean())
                    j+=stride_len[iteration]
                temp_features.append(temp)
            self.features.append(temp_features)
    
    
    def _get_m(self,stride_len):
        '''
        stride_len: 
        based on stride length,& num_features, we calculate possible filter size 
        '''
        m = self.z -(self.num_features)*stride_len
        return m
    
    
    def result_features(self):
        '''
        if multiple filters were added, takes the average result
        '''
        ans =[]
        for timeseries in range(len(self.features[0])):
            temp =[]
            for feature in range(len(self.features[0][0])):
                val = np.mean([self.features[filter][timeseries][feature] for filter in range(len(self.features))])
                temp.append(val)
            ans.append(temp)
        return ans
        
        

In [7]:
def transform_data(data, num_features, stride_len, num_filters):
    data_transformed = reservoir_features(data ,num_features = num_features)
    #normalize
    data_transformed.normalize()
    #create 2 filters
    data_transformed.filters(stride_len = stride_len, num_filters = num_filters)
    data_transformed = data_transformed.result_features()
    return data_transformed

In [8]:
X_train_transformed= transform_data(X_3d, 40, [3], 1)

filter of size  9 * 24 was created



In [9]:
X_test_transformed=transform_data(X_test_3d, 40, [3], 1)

filter of size  9 * 24 was created



In [10]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

In [11]:
forest = RandomForestClassifier()

# Create the grid:
gs_rf = GridSearchCV(forest, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

# Fit using grid search:
gs_rf.fit(X_train_transformed, y)

# Print best accuracy and best parameters:
print('Best accuracy: %.3f' % gs_rf.best_score_)
print('\nBest params:\n', gs_rf.best_params_)

Fitting 3 folds for each of 288 candidates, totalling 864 fits
Best accuracy: 0.571

Best params:
 {'bootstrap': True, 'max_depth': 110, 'max_features': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 100}


In [12]:
clf_test_pred=gs_rf.predict(X_test_transformed)
clf_test_acc_score = accuracy_score(y_test, clf_test_pred)
print(clf_test_acc_score)

0.5633333333333334


In [13]:
roc_auc_score(y_test, gs_rf.predict_proba(X_test_transformed), multi_class='ovr')

0.9536226851851852

In [14]:
clf_C= RandomForestClassifier(bootstrap= True, max_depth=100, max_features=2,min_samples_leaf=4, min_samples_split=8,
                              n_estimators=200)

clf_C.fit(X_test_transformed, y_test)

RandomForestClassifier(max_depth=100, max_features=2, min_samples_leaf=4,
                       min_samples_split=8, n_estimators=200)

In [15]:
clf_test_pred=clf_C.predict(X_test_transformed)
clf_test_acc_score = accuracy_score(y_test, clf_test_pred)
print(clf_test_acc_score)

0.9233333333333333


In [16]:
roc_auc_score(y_test, clf_C.predict_proba(X_test_transformed), multi_class='ovr')

0.9985648148148148